In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# stacked generalization with neural net meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from numpy import argmax
from os import makedirs
from keras.models import Sequential
from matplotlib import pyplot

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot
from sklearn.model_selection import LeaveOneOut, GridSearchCV, KFold, train_test_split
from sklearn import svm
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score, classification_report, matthews_corrcoef, roc_auc_score
from tensorflow.keras import regularizers

In [ ]:
pd_speech_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PD Speech Features/gender_FG_v2.csv')
pd_speech_features.head()
print('The shape of the matrix is :', pd_speech_features.shape)
#pd_speech_features['patient/healthy count'] = 1
print(pd_speech_features['gender'].value_counts())
pd_speech_features[['gender', 'class']] = pd_speech_features[['gender', 'class']].astype('category') #categoricals
pd_speech_features.dtypes

In [ ]:
df=pd_speech_features
train_df=df.values
x=train_df[:,:-1]
y=to_categorical(train_df[:,-1], num_classes=2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x= sc.fit_transform(x)

In [ ]:
#newly added
kfold=KFold(10, shuffle=True, random_state=0)

#no of individual models
n_members = 3
      
acc=[]
pr=[]
rc=[]
f1=[]
auc=[]
kappa=[]

fold_count=0
for trids,tstids in kfold.split(x):
      fold_count=fold_count+1
      trainX,testX=x[trids,:],x[tstids,:]
      trainY,testY=y[trids,:],y[tstids,:]

      #[samples, time steps, features]
      n_features=trainX.shape[1]
      trainX = np.array(trainX)
      trainY = np.array(trainY)
      testX = np.array(testX)
      testY = np.array(testY)

      print(trainX.shape)
      print(trainY.shape)
      print(testX.shape)
      print(testY.shape)
      
      #makedirs('models')
      
      # load models
      filename = '/content/drive/MyDrive/Colab Notebooks/PD Speech Features/models/ensemble_model_fold_'+ str(fold_count) +'.h5'
      model = load_model(filename)
      
      [accr, precision, recall, f, a, k] = eval_model(model,testX,testY)
      
      acc.append(accr)
      pr.append(precision)
      rc.append(recall)
      f1.append(f)
      auc.append(a)
      kappa.append(k)
      
      print(acc)
      print(f1)
      print(auc)

(680, 71)
(680, 2)
(76, 71)
(76, 2)
Accuracy: 0.947368
Precision: 0.982456
Recall: 0.949153
F1 score: 0.965517
Cohens kappa: 0.854545
ROC AUC: 0.992024
[[16  1]
 [ 3 56]]
[0.9473684210526315]
[0.9655172413793103]
[0.992023928215354]
(680, 71)
(680, 2)
(76, 71)
(76, 2)
Accuracy: 0.934211
Precision: 0.946429
Recall: 0.963636
F1 score: 0.954955
Cohens kappa: 0.833040
ROC AUC: 0.932684
[[18  3]
 [ 2 53]]
[0.9473684210526315, 0.9342105263157895]
[0.9655172413793103, 0.9549549549549549]
[0.992023928215354, 0.9326839826839827]
(680, 71)
(680, 2)
(76, 71)
(76, 2)
Accuracy: 0.960526
Precision: 0.965517
Recall: 0.982456
F1 score: 0.973913
Cohens kappa: 0.892857
ROC AUC: 0.989843
[[17  2]
 [ 1 56]]
[0.9473684210526315, 0.9342105263157895, 0.9605263157894737]
[0.9655172413793103, 0.9549549549549549, 0.9739130434782608]
[0.992023928215354, 0.9326839826839827, 0.989843028624192]
(680, 71)
(680, 2)
(76, 71)
(76, 2)
Accuracy: 0.973684
Precision: 0.969231
Recall: 1.000000
F1 score: 0.984375
Cohens kapp

In [ ]:
from statistics import mean
print('acc: %f',mean(acc))
print('f1: %f',mean(f1))
print('auc: %f',mean(auc))
print('pr: %f',mean(pr))
print('rc: %f',mean(rc))
print('kappa: %f',mean(kappa))


acc: %f 0.9509824561403508
f1: %f 0.9670000804199732
auc: %f 0.9715638754017141
pr: %f 0.9578433438914382
rc: %f 0.9766562617677693
kappa: %f 0.8699180852484752


In [ ]:
def eval_model(model,testX,testY):
   # predict probabilities for test set
   y_prob = predict_stacked_model(model, testX)
   # predict crisp classes for test set
   y_classes=np.argmax(y_prob,axis=1)
   act_Y=np.argmax(testY,axis=1)
   
   # accuracy: (tp + tn) / (p + n)
   accuracy = accuracy_score(act_Y, y_classes)
   print('Accuracy: %f' % accuracy)
   # precision tp / (tp + fp)
   precision = precision_score(act_Y, y_classes)
   print('Precision: %f' % precision)
   # recall: tp / (tp + fn)
   recall = recall_score(act_Y, y_classes)
   print('Recall: %f' % recall)
   # f1: 2 tp / (2 tp + fp + fn)
   f1 = f1_score(act_Y, y_classes)
   print('F1 score: %f' % f1)
 
   # kappa
   kappa = cohen_kappa_score(act_Y, y_classes)
   print('Cohens kappa: %f' % kappa)
   # ROC AUC
   auc = roc_auc_score(testY, y_prob)
   print('ROC AUC: %f' % auc)
   # confusion matrix
   matrix = confusion_matrix(act_Y, y_classes)
   print(matrix)
   
   return [accuracy, precision, recall, f1, auc, kappa]

In [ ]:
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)